# 🎬 PHOTOGRAPHY PRODUCTION — Pipeline Cinématique

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 FRÉGATE 04_PHOTOGRAPHY — PRODUCTION PIPELINE                 ║
║              Tracking Caméra + Éclairage Cinématique Automatisés             ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Mission**: Configurer caméras et lumières selon le PRODUCTION_PLAN.JSON

---

## 📦 Phase 1: Setup Environnement

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration EXODUS
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/04_PHOTOGRAPHY_WING"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"
AI_MODELS = f"{DRIVE_ROOT}/EXODUS_AI_MODELS"

# Paths
IN_SCENE = f"{UNIT_ROOT}/IN_SCENE"
OUT_CAMERA = f"{UNIT_ROOT}/OUT_CAMERA"

import sys
sys.path.insert(0, CODEBASE)

print("═" * 60)
print("  FRÉGATE 04_PHOTOGRAPHY — EXODUS PRODUCTION")
print("═" * 60)
print(f"✓ Drive Root: {DRIVE_ROOT}")
print(f"✓ Unit Root: {UNIT_ROOT}")

In [ ]:
# Install dependencies
!pip install -q scipy numpy matplotlib

# Download Blender 4.0 if not present
import os
from pathlib import Path

BLENDER_DIR = f"{AI_MODELS}/blender-4.0.0-linux-x64"
BLENDER_BIN = f"{BLENDER_DIR}/blender"

if not Path(BLENDER_BIN).exists():
    print("⏳ Téléchargement Blender 4.0...")
    !wget -q https://download.blender.org/release/Blender4.0/blender-4.0.0-linux-x64.tar.xz -O /tmp/blender.tar.xz
    !mkdir -p {AI_MODELS}
    !tar -xf /tmp/blender.tar.xz -C {AI_MODELS}/
    !rm /tmp/blender.tar.xz
    print("✓ Blender 4.0 installé")
else:
    print("✓ Blender 4.0 déjà présent")

# Verify
!{BLENDER_BIN} --version | head -n 1

## 📋 Phase 2: Vérification Inputs

In [ ]:
import json

# Lister les fichiers IN_SCENE
print("=== Fichiers IN_SCENE ===")
in_files = list(Path(IN_SCENE).glob("*"))
blend_files = [f for f in in_files if f.suffix == ".blend"]
json_files = [f for f in in_files if f.suffix == ".json"]

print(f"\n📁 Fichiers .blend: {len(blend_files)}")
for f in blend_files:
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"   - {f.name} ({size_mb:.1f} MB)")

print(f"\n📄 Fichiers .json: {len(json_files)}")
for f in json_files:
    print(f"   - {f.name}")

In [ ]:
# === CONFIGURATION ===
# Modifier ces valeurs selon votre projet

PRODUCTION_PLAN = "PRODUCTION_PLAN.JSON"  # Nom du fichier plan

# =====================

plan_path = Path(IN_SCENE) / PRODUCTION_PLAN

if plan_path.exists():
    with open(plan_path, 'r') as f:
        plan = json.load(f)
    
    print(f"✓ PRODUCTION_PLAN chargé: {plan_path.name}")
    print(f"\n=== Contenu du Plan ===")
    print(f"Projet: {plan.get('project', 'N/A')}")
    print(f"Scènes: {len(plan.get('scenes', []))}")
    
    for scene in plan.get('scenes', []):
        sid = scene.get('scene_id', '?')
        cam = scene.get('camera', {})
        light = scene.get('lighting', {})
        
        print(f"\n  Scene {sid}:")
        print(f"    Camera: style={cam.get('style', 'N/A')}, movement={cam.get('movement', 'N/A')}")
        print(f"    Cuts: {len(cam.get('cuts', []))}")
        print(f"    Lighting: style={light.get('style', 'N/A')}, intensity={light.get('intensity', 1.0)}")
else:
    print(f"✗ PRODUCTION_PLAN introuvable: {plan_path}")
    print("  Placez votre PRODUCTION_PLAN.JSON dans IN_SCENE/")

## 🎬 Phase 3: Exécution Pipeline

In [ ]:
# Dry-run d'abord pour valider
!python {CODEBASE}/EXO_04_PHOTOGRAPHY.py \
    --drive-root {DRIVE_ROOT} \
    --production-plan {PRODUCTION_PLAN} \
    --blender-path {BLENDER_BIN} \
    --dry-run -v

In [ ]:
# === EXÉCUTION PRODUCTION ===
# Décommentez pour lancer le traitement complet

# !python {CODEBASE}/EXO_04_PHOTOGRAPHY.py \
#     --drive-root {DRIVE_ROOT} \
#     --production-plan {PRODUCTION_PLAN} \
#     --blender-path {BLENDER_BIN} \
#     -v

In [ ]:
# Traiter une scène spécifique
SCENE_ID = 1  # Modifier selon besoin

# !python {CODEBASE}/EXO_04_PHOTOGRAPHY.py \
#     --drive-root {DRIVE_ROOT} \
#     --production-plan {PRODUCTION_PLAN} \
#     --blender-path {BLENDER_BIN} \
#     --scene-id {SCENE_ID} \
#     -v

## 📊 Phase 4: Inspection Outputs

In [ ]:
# Lister les outputs générés
print("=== Fichiers OUT_CAMERA ===")
out_files = list(Path(OUT_CAMERA).glob("*"))

blend_out = [f for f in out_files if f.suffix == ".blend"]
json_out = [f for f in out_files if f.suffix == ".json"]

print(f"\n📁 Scènes prêtes (.blend): {len(blend_out)}")
for f in sorted(blend_out):
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"   - {f.name} ({size_mb:.1f} MB)")

print(f"\n📄 Données exportées (.json): {len(json_out)}")
for f in sorted(json_out):
    print(f"   - {f.name}")

In [ ]:
# Inspecter le rapport de production
report_path = Path(OUT_CAMERA) / "photography_report.json"

if report_path.exists():
    with open(report_path, 'r') as f:
        report = json.load(f)
    
    print("=== Photography Report ===")
    print(f"Status: {report.get('status')}")
    print(f"Timestamp: {report.get('timestamp')}")
    
    summary = report.get('summary', {})
    print(f"\nScènes: {summary.get('scenes_processed', 0)}/{summary.get('scenes_total', 0)} traitées")
    
    if summary.get('scenes_failed', 0) > 0:
        print(f"⚠ Échecs: {summary.get('scenes_failed')}")
    
    print("\nDétail:")
    for scene in report.get('scenes', []):
        status = "✓" if scene.get('success') else "✗"
        print(f"  {status} Scene {scene.get('scene_id')}: "
              f"cam={scene.get('camera', {}).get('style')}, "
              f"light={scene.get('lighting', {}).get('style')}")
else:
    print("Rapport non trouvé - lancez d'abord le pipeline")

In [ ]:
# Inspecter les données caméra d'une scène
INSPECT_SCENE_ID = 1

camera_data_path = Path(OUT_CAMERA) / f"camera_data_{INSPECT_SCENE_ID}.json"

if camera_data_path.exists():
    with open(camera_data_path, 'r') as f:
        cam_data = json.load(f)
    
    print(f"=== Camera Data Scene {INSPECT_SCENE_ID} ===")
    print(f"Frame Range: {cam_data.get('frame_range')}")
    
    cam = cam_data.get('camera', {})
    print(f"\nCaméra:")
    print(f"  Name: {cam.get('name')}")
    print(f"  Style: {cam.get('style')}")
    print(f"  FOV: {cam.get('fov')}°")
    print(f"  Location: {cam.get('location')}")
    
    target = cam_data.get('target', {})
    print(f"\nTarget:")
    print(f"  Location: {target.get('location')}")
    
    ops = cam_data.get('operations', [])
    print(f"\nOpérations: {len(ops)}")
    for op in ops[:10]:  # Premiers 10
        print(f"  - {op.get('action')}: {op.get('message', op.get('type', 'N/A'))}")
else:
    print(f"Données caméra non trouvées pour scene {INSPECT_SCENE_ID}")

## 🎨 Phase 5: Prévisualisation (Optionnel)

In [ ]:
# Générer un aperçu de la trajectoire caméra
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def plot_camera_trajectory(camera_data):
    """Visualise la trajectoire caméra en 3D."""
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Position caméra
    cam_loc = camera_data.get('camera', {}).get('location', [0, 0, 0])
    ax.scatter(*cam_loc, c='blue', s=100, marker='^', label='Caméra')
    
    # Target
    target_loc = camera_data.get('target', {}).get('location', [0, 0, 0])
    ax.scatter(*target_loc, c='red', s=100, marker='o', label='Target')
    
    # Ligne de vue
    ax.plot([cam_loc[0], target_loc[0]], 
            [cam_loc[1], target_loc[1]], 
            [cam_loc[2], target_loc[2]], 'g--', alpha=0.5, label='Ligne de vue')
    
    # Extraire les keyframes si disponibles
    keyframes = []
    for op in camera_data.get('operations', []):
        if op.get('action') == 'keyframe' and op.get('location'):
            keyframes.append(op['location'])
    
    if keyframes:
        kf_array = np.array(keyframes)
        ax.plot(kf_array[:, 0], kf_array[:, 1], kf_array[:, 2], 
                'b-', linewidth=2, alpha=0.7, label='Trajectoire')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f"Trajectoire Caméra - Scene {camera_data.get('scene_id')}")
    ax.legend()
    
    plt.tight_layout()
    plt.show()

# Usage
# if camera_data_path.exists():
#     plot_camera_trajectory(cam_data)

## 📝 Phase 6: Export pour U05

In [ ]:
# Résumé des outputs pour la prochaine unité
print("═" * 60)
print("  OUTPUTS POUR U05_ALCHEMIST_LAB")
print("═" * 60)

ready_scenes = list(Path(OUT_CAMERA).glob("scene_ready_*.blend"))

if ready_scenes:
    print(f"\n✓ {len(ready_scenes)} scène(s) prête(s) au rendu:")
    for scene in sorted(ready_scenes):
        print(f"  → {scene.name}")
    
    print(f"\nCopier vers U05:")
    print(f"  Source: {OUT_CAMERA}/scene_ready_*.blend")
    print(f"  Dest:   {DRIVE_ROOT}/05_ALCHEMIST_LAB/IN_RENDER/")
else:
    print("\n✗ Aucune scène prête - lancez d'abord le pipeline")

---
## ✅ Checklist Production

- [ ] Environment monté
- [ ] Blender 4.0 installé
- [ ] PRODUCTION_PLAN.JSON présent
- [ ] Fichiers .blend d'environnement présents
- [ ] Dry-run validé
- [ ] Pipeline exécuté
- [ ] Rapport vérifié
- [ ] Outputs prêts pour U05